# 確率ロボティクス2017第9回

上田隆一

2017年11月15日@千葉工業大学

## 今日やること

* 強化学習（古典的なもの）
* 参考: [Sutton and Barto, 三上, 皆川訳: 強化学習, 森北出版, 2000.](https://www.amazon.co.jp/dp/4627826613)
  * [ウェブ版 ](http://webdocs.cs.ualberta.ca/~sutton/book/ebook/the-book.html)
  * [そろそろ第2版が出るらしい](http://www.freetechbooks.com/reinforcement-learning-an-introduction-second-edition-draft-t1282.html)
  
## 強化学習（reinforcement learning）

* 機械学習の一種
  * 起源: 動物の心理学研究から[宮崎2005]
    * 餌にありつけるような/危険を避けるような行動の学習
  * 最近ANN（人工ニューラルネットワーク）との組み合わせで脚光を浴びている
  
## 基本的な強化学習の問題

* 有限MDPでの行動決定の問題に一つ制限を与える
  * 状態遷移と報酬が、行動しないと分からない。
* 問題の定義
  * 時刻: $\mathcal{T} = \{t | t = 0,1,2,\dots,T \}$
  * 状態: $\mathcal{S} = \{s_i | i = 1,2,3,\dots,N\}$
    * うち、いくつかは終端状態の集合$\mathcal{S}_\text{f}$に含まれる
  * 行動: $\mathcal{A} = \{a_j | j = 1,2,3,\dots,M\}$
  * 状態遷移や報酬は時不変だが自明でない
    * ある確率分布に従って状態遷移する
    * ある法則に従って報酬が与えられる 

## 何を求めたいのか

* 「最適方策」を求める
* 方策（二種類）
  * 決定論的方策: $\pi : \mathcal{S} \to \mathcal{A}$
    * 状態が決まると行動が決まるもの
  * 確率的な方策: $\pi : \mathcal{S}\times \mathcal{A} \to \Re$
    * 状態$s$において行動$a$を選択する確率
* 最適方策は決定論的になる（理由は次のページに）
  * $\pi^* : \mathcal{S} \to \mathcal{A}$
  
## 問題を解く道具

* ベルマン方程式が成り立つ
  * $V^*(s) = \max_{a \in \mathcal{A} } \sum_{s' \in \mathcal{S}}\mathcal{P}_{ss'}^a [V^*(s') +\mathcal{R}_{ss'}^a ]$
    * $V^*$: 最適状態価値関数
    * 左辺の「$\max_{a \in \mathcal{A} }$」を満たすものが最適方策$\pi^*$
* エージェントの「経験」
  * 状態遷移: ある状態$s$で行動$a$を選択したら状態$s'$に遷移して報酬を得た
  * たくさん行動すれば統計的な性質が得られる
  * たくさん行動しなければ統計的な性質が得られない
  
## 問題の難しさ

* 価値関数に停留点が一つでもあると、エージェントがその場に留まってしまう
* 膨大な試行が必要